In [1]:
import os
import sys
import random
import pandas as pd
import numpy as np
from scipy.linalg import toeplitz
from copy import copy
import matplotlib.pyplot as plt
%matplotlib inline

# Geniuses that worked on hypertools did not update certain package and thus it produces warnings (they break jupyter lab)
import warnings
warnings.filterwarnings("ignore")

# Comment out if you don't want to see all of the values being printed (i.e. default)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

current_dir = os.getcwd()
# utils_path = os.path.join(current_dir, '..', 'utils')
utils_path = os.path.join(current_dir, '../')
utils_abs_path = os.path.abspath(utils_path)
if utils_abs_path not in sys.path:
    sys.path.append(utils_abs_path)

import utils.get_data as get_data
# from impute_methods import *
from utils.impute_methods import impute_linear_interpolation

DATA_PATH = get_data.get_dataset_abspath()

training_setA_path = DATA_PATH + 'training_setA'
training_setB_path = DATA_PATH + 'training_setB'

In [2]:
import os
import sys
import random
import pandas as pd
import numpy as np
from scipy.linalg import toeplitz
from copy import copy
import matplotlib.pyplot as plt
%matplotlib inline

# Geniuses that worked on hypertools did not update certain package and thus it produces warnings (they break jupyter lab)
import warnings
warnings.filterwarnings("ignore")

# Comment out if you don't want to see all of the values being printed (i.e. default)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

current_dir = os.getcwd()
# utils_path = os.path.join(current_dir, '..', 'utils')
utils_path = os.path.join(current_dir, '../')
utils_abs_path = os.path.abspath(utils_path)
if utils_abs_path not in sys.path:
    sys.path.append(utils_abs_path)

import utils.get_data as get_data
# from impute_methods import *
from utils.impute_methods import impute_linear_interpolation

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class TransformerTimeSeries(nn.Module):
    def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=2, dropout=0.2):
        super(TransformerTimeSeries, self).__init__()
        self.encoder = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x.transpose(0, 1))
        x = self.decoder(x.transpose(0, 1))
        return x.squeeze(-1)

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


class TransformerTimeSeries(nn.Module):
    def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=2, dropout=0.2):
        super(TransformerTimeSeries, self).__init__()

        self.encoder = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x.transpose(0, 1))
        x = self.decoder(x)
        return x.squeeze(-1)
        
        


In [5]:
# Load and preprocess data
dataset, patient_id_map = get_data.get_dataset()

   20337
   40337
Dataset loaded into a MultiIndex DataFrame.


In [6]:
dataset.shape

(1552210, 41)

In [7]:
# First lets experiment with only raw data 
# We have to however impute NaN values since Neural Networks can't (natively) handle them

columns_to_linearly_interpolate = [
    'HR', 'O2Sat', 'SBP', 'MAP', 'DBP', 'Resp'
]

# Feel free to omit this (EXPERIMENTAL)
# Normilize the dataset
if True:
    # Check if multiindex_df is indeed a MultiIndex DataFrame
    if isinstance(dataset.index, pd.MultiIndex):
        # Exclude 'SepsisLabel' from normalization
        features_to_normalize = dataset.columns.difference(['SepsisLabel'])

        # Normalize each patient's data
        # This will apply z-score normalization per patient per feature, excluding 'SepsisLabel'
        normalized_data = dataset[features_to_normalize].groupby(level=0).transform(
            lambda x: (x - x.mean()) / x.std())

        # Optionally fill NaN values if they are created by division by zero in cases where std is zero
        normalized_data = normalized_data.fillna(0)

        # Merge normalized data with the 'SepsisLabel' column
        dataset = pd.concat([normalized_data, dataset['SepsisLabel']], axis=1)
    else:
        print("The dataframe does not have a MultiIndex as expected.")

# Linear Interpolation
print("Linearly interpolating:")
for col in columns_to_linearly_interpolate:
    if col != 'SepsisLabel':  # Ensure we do not interpolate 'SepsisLabel'
        dataset = impute_linear_interpolation(dataset, col)
        print(col)
print("Done")

Linearly interpolating:
HR
O2Sat
SBP
MAP
DBP
Resp
Done


In [8]:
dataset.shape

(1552210, 41)

In [9]:
def add_nan_indicators(df):
    for column in df.columns:
        df[column + '_nan'] = df[column].isna().astype(int)
    return df

In [10]:
def downsample(X, y):
    index_0 = np.where(y == 0)[0]
    index_1 = np.where(y == 1)[0]
    print(index_0, index_1)

    if len(index_0) > len(index_1):
        index_0 = np.random.choice(index_0, size=len(index_1), replace=False)

    balanced_indices = np.concatenate([index_0, index_1])
    np.random.shuffle(balanced_indices)

    x_balanced = X.iloc[balanced_indices]
    y_balanced = y.iloc[balanced_indices]

    return x_balanced, y_balanced
    

In [125]:
import torch
from torchsummary import summary
model = TimeSeriesTransformer(input_dim=X_train.shape[1])
summary(model, input_size=(1, 64))

ModuleNotFoundError: No module named 'torchsummary'

In [11]:
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} for training.")

Using cpu for training.


In [12]:
X = dataset.drop('SepsisLabel', axis=1)
X = add_nan_indicators(X)
y = dataset['SepsisLabel']

for patient_id in X.index.get_level_values('patient_id').unique():
    print(patient_id)
    break

a = 0
for patient_id, patient_data in X.groupby(level='patient_id'):
    print(patient_data.shape)
    if a > 10:
        break
    a += 1


1.0
(28, 80)
(34, 80)
(37, 80)
(29, 80)
(47, 80)
(26, 80)
(44, 80)
(56, 80)
(37, 80)
(32, 80)
(26, 80)
(27, 80)


In [22]:
from sklearn.preprocessing import StandardScaler

def prepare_patient_data(patient_data, max_length):
    # Standardizing the data
    scaler = StandardScaler()
    features = scaler.fit_transform(patient_data)
    # Padding
    padded_features = np.zeros((max_length, features.shape[1]))
    sequence_length = min(max_length, features.shape[0])
    padded_features[:sequence_length] = features[:sequence_length]
    return torch.tensor(padded_features, dtype=torch.float32), sequence_length


In [38]:
X = df.drop('SepsisLabel', axis=1)
print(X.shape)

max_length = X.groupby('patient_id').size().max()
print("Max length (inputs will be padded to): ", max_length)

patient_ids = X.index.get_level_values('patient_id').unique()
for patient_id in patient_ids:
    max_length_patient = X.loc[patient_id].shape[0]
    if max_length_patient == max_length:
        print(patient_id, max_length_patient)

(1552210, 40)
Max length (inputs will be padded to):  336
457.0 336
1126.0 336
7228.0 336
12849.0 336
14201.0 336
23823.0 336
25080.0 336
26436.0 336
33001.0 336
36236.0 336


In [ ]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import KFold

# Assuming TransformerTimeSeries and other necessary classes/functions are defined

df = dataset
X = df.drop('SepsisLabel', axis=1)
X = add_nan_indicators(X)
y = df['SepsisLabel']

# Find the maximum sequence length for padding
max_length = X.groupby('patient_id').size().max()
print("Max length (inputs will be padded to): ", max_length)

patient_ids = X.index.get_level_values('patient_id').unique()

# KFold configuration
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize class weights
classes = np.array([0, 1])
class_weights = compute_class_weight('balanced', classes=classes, y=y.to_numpy())
class_weights_tensor = torch.tensor(class_weights[1], dtype=torch.float)

# Training loop with K-Fold Cross-Validation
num_epochs = 10
fold_idx = 1

for train_index, val_index in kf.split(patient_ids):
    train_ids, val_ids = patient_ids[train_index], patient_ids[val_index]

    # Initialize model, criterion, and optimizer for each fold
    input_dim = X.shape[1]
    model = TransformerTimeSeries(input_dim=input_dim)
    criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights_tensor)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print(f"Starting training for fold {fold_idx}")
    for epoch in range(num_epochs):
        model.train()
        train_loss, train_correct, train_total = 0, 0, 0
        train_preds, train_targets = [], []
        
        start_time = time.time()
        
        for i, patient_id in enumerate(train_ids):
            start_batch = time.time()
            
            try:
                patient_data = X.loc[patient_id]
                X_train, sequence_length = prepare_patient_data(patient_data, max_length)
                y_train = torch.tensor(y.loc[patient_id].values, dtype=torch.float32)
                
                optimizer.zero_grad()
                outputs = model(X_train.unsqueeze(0))
                loss = criterion(outputs[:sequence_length].squeeze(), y_train[:sequence_length])
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                predicted_labels = torch.round(torch.sigmoid(outputs[:sequence_length].squeeze()))
                train_correct += (predicted_labels == y_train[:sequence_length]).sum().item()
                train_total += sequence_length

                train_preds.extend(predicted_labels.tolist())
                train_targets.extend(y_train[:sequence_length].tolist())
            
            except Exception as e:
                print(f"Error processing patient ID {patient_id}: {e}")
            
            end_batch = time.time()
            elapsed_time = end_batch - start_time
            batches_done = i + 1
            total_batches = len(train_ids)
            time_per_batch = elapsed_time / batches_done
            estimated_time_remaining = (total_batches - batches_done) * time_per_batch
            
            print(f'Processed {batches_done}/{total_batches} patients ({100.0 * batches_done / total_batches:.2f}%), ' +
                  f'Estimated time remaining: {estimated_time_remaining / 60:.2f} minutes', end='\r')

        print()  # Ensure newline after progress updates
        train_accuracy = train_correct / train_total
        train_precision = precision_score(train_targets, train_preds)
        train_recall = recall_score(train_targets, train_preds)
        train_f1 = f1_score(train_targets, train_preds)
        print(f'Epoch {epoch+1}, Fold {fold_idx}, Avg Training Loss: {train_loss / len(train_ids)}, Training Metrics: Acc: {train_accuracy}, Precision: {train_precision}, Recall: {train_recall}, F1: {train_f1}')

    # Validation after all epochs in a fold
    validate_model(model, val_ids, X, y, max_length, criterion, fold_idx)
    fold_idx += 1

def validate_model(model, val_ids, X, y, max_length, criterion, fold_idx):
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    val_preds, val_targets = [], []
    start_val_time = time.time()

    with torch.no_grad():
        for i, patient_id in enumerate(val

In [ ]:
import time
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

df = dataset
X = df.drop('SepsisLabel', axis=1)
X = add_nan_indicators(X)
y = df['SepsisLabel']

# Find the maximum sequence length for padding
# Yes it's really high, 336, consider making it larger to accommodate actual test set
max_length = X.groupby('patient_id').size().max()
print("Max length (inputs will be padded to): ", max_length)

patient_ids = X.index.get_level_values('patient_id').unique()
train_ids, val_ids = train_test_split(patient_ids, test_size=0.2, random_state=42)

# Initialize model, criterion, and optimizer
input_dim = X.shape[1]
model = TransformerTimeSeries(input_dim=input_dim)

classes = np.array([0, 1])
class_weights = compute_class_weight('balanced', classes=classes, y=y.to_numpy())
class_weights_tensor = torch.tensor(class_weights[1], dtype=torch.float)  # Weight for the positive class

# Initialize BCEWithLogitsLoss with pos_weight
criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights_tensor)

optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    train_loss, train_correct, train_total = 0, 0, 0
    train_preds, train_targets = [], []
    
    start_time = time.time()  # Start time for the epoch
    
    for i, patient_id in enumerate(train_ids):
        start_batch = time.time()  # Start time for the batch
        
        try:
            patient_data = X.loc[patient_id]
            X_train, sequence_length = prepare_patient_data(patient_data, max_length)
            y_train = torch.tensor(y.loc[patient_id].values, dtype=torch.float32)
            
            optimizer.zero_grad()
            outputs = model(X_train.unsqueeze(0))
            loss = criterion(outputs[:sequence_length].squeeze(), y_train[:sequence_length])
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            predicted_labels = torch.round(torch.sigmoid(outputs[:sequence_length].squeeze()))
            train_correct += (predicted_labels == y_train[:sequence_length]).sum().item()
            train_total += sequence_length

            train_preds.extend(predicted_labels.tolist())
            train_targets.extend(y_train[:sequence_length].tolist())
        
        except Exception as e:
            print(f"Error processing patient ID {patient_id}: {e}")
        
        # Progress and time estimation
        end_batch = time.time()
        elapsed_time = end_batch - start_time
        batches_done = i + 1
        total_batches = len(train_ids)
        time_per_batch = elapsed_time / batches_done
        estimated_time_remaining = (total_batches - batches_done) * time_per_batch
        
        print(f'Processed {batches_done}/{total_batches} patients ({100.0 * batches_done / total_batches:.2f}%), ' +
              f'Estimated time remaining: {estimated_time_remaining / 60:.2f} minutes', end='\r')

    train_accuracy = train_correct / train_total
    train_precision = precision_score(train_targets, train_preds)
    train_recall = recall_score(train_targets, train_preds)
    train_f1 = f1_score(train_targets, train_preds)

    print(f'Epoch {epoch+1}, Avg Training Loss: {train_loss / len(train_ids)}, Training Metrics: Acc: {train_accuracy}, Precision: {train_precision}, Recall: {train_recall}, F1: {train_f1}')


    # Validation phase
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    val_preds, val_targets = [], []
    start_val_time = time.time()

    with torch.no_grad():
        for i, patient_id in enumerate(val_ids):
            start_batch = time.time()

            try:
                patient_data = X.loc[patient_id]
                X_val, sequence_length = prepare_patient_data(patient_data, max_length)
                y_val = torch.tensor(y.loc[patient_id].values, dtype=torch.float32)

                val_outputs = model(X_val.unsqueeze(0))
                v_loss = criterion(val_outputs[:sequence_length].squeeze(), y_val[:sequence_length])
                val_loss += v_loss.item()

                val_predicted_labels = torch.round(torch.sigmoid(val_outputs[:sequence_length].squeeze()))
                val_correct += (val_predicted_labels == y_val[:sequence_length]).sum().item()
                val_total += sequence_length

                val_preds.extend(val_predicted_labels.tolist())
                val_targets.extend(y_val[:sequence_length].tolist())
            
            except Exception as e:
                print(f"Error processing patient ID {patient_id}: {e}")

            # Progress and time estimation for validation
            end_batch = time.time()
            elapsed_time = end_batch - start_val_time
            batches_done = i + 1
            total_batches = len(val_ids)
            time_per_batch = elapsed_time / batches_done
            estimated_time_remaining = (total_batches - batches_done) * time_per_batch
            
            print(f'Validation: Processed {batches_done}/{total_batches} patients ({100.0 * batches_done / total_batches:.2f}%), ' +
                  f'Estimated time remaining: {estimated_time_remaining / 60:.2f} minutes', end='\r')

    val_accuracy = val_correct / val_total
    val_precision = precision_score(val_targets, val_preds)
    val_recall = recall_score(val_targets, val_preds)
    val_f1 = f1_score(val_targets, val_preds)

    # Print epoch summary
    print(f'Epoch {epoch+1}, Avg Training Loss: {train_loss / len(train_ids)}, Training Metrics: Acc: {train_accuracy}, Precision: {train_precision}, Recall: {train_recall}, F1: {train_f1}')
    print(f'Avg Validation Loss: {val_loss / len(val_ids)}, Validation Metrics: Acc: {val_accuracy}, Precision: {val_precision}, Recall: {val_recall}, F1: {val_f1}')


Max length (inputs will be padded to):  336
tensor(27.8014)


TransformerTimeSeries(
  (encoder): Linear(in_features=80, out_features=64, bias=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Linear(in_features=64, out_features=1, bias=True)
)

Epoch 1, Avg Training Loss: 1.2415898634351488, Training Metrics: Acc: 0.8017604933439951, Precision: 0.018793903703289472, Recall: 0.19465359768322565, F1: 0.0342782271668092


TransformerTimeSeries(
  (encoder): Linear(in_features=80, out_features=64, bias=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Linear(in_features=64, out_features=1, bias=True)
)

Epoch 1, Avg Training Loss: 1.2415898634351488, Training Metrics: Acc: 0.8017604933439951, Precision: 0.018793903703289472, Recall: 0.19465359768322565, F1: 0.0342782271668092
Avg Validation Loss: 1.1857613008774088, Validation Metrics: Acc: 0.7753053328135722, Precision: 0.026631315363087348, Recall: 0.33046974958874065, F1: 0.04929049494963265


...

TransformerTimeSeries(
  (encoder): Linear(in_features=80, out_features=64, bias=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Linear(in_features=64, out_features=1, bias=True)
)

Epoch 3, Avg Training Loss: 1.1536127951813997, Training Metrics: Acc: 0.8839928137212506, Precision: 0.03455956645540553, Recall: 0.20115838716863443, F1: 0.058985296134928054


TransformerTimeSeries(
  (encoder): Linear(in_features=80, out_features=64, bias=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Linear(in_features=64, out_features=1, bias=True)
)

Epoch 3, Avg Training Loss: 1.1536127951813997, Training Metrics: Acc: 0.8839928137212506, Precision: 0.03455956645540553, Recall: 0.20115838716863443, F1: 0.058985296134928054
Avg Validation Loss: 1.1453850496713631, Validation Metrics: Acc: 0.939635759135822, Precision: 0.058623902049507584, Recall: 0.16103089014805338, F1: 0.08595541245914436


TransformerTimeSeries(
  (encoder): Linear(in_features=80, out_features=64, bias=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Linear(in_features=64, out_features=1, bias=True)
)